# Training models

In [ ]:
# download and extract the glycine dataset (B3LYP energies and gradients) from “Full-dimensional, ab initio potential energy surface for glycine with characterization of stationary points and zero-point energy calculations by means of diffusion Monte Carlo and semiclassical dynamics“, Conte et al.,  J. Chem. Phys. 153, 244301 (2020)
!wget https://scholarblogs.emory.edu/bowman/files/2021/03/Gly70099.zip 
!unzip Gly70099.zip

In [ ]:
# data processing
!sed -n '2~12p' Gly70099\ copy.xyz > en.dat
!awk '{print $1,$2,$3,$4}' Gly70099\ copy.xyz > xyz.dat
!awk '{if (NR % 12 == 1) print $1; else print $5,$6,$7}' Gly70099\ copy.xyz > grad.dat

In [ ]:
import mlatom as ml
import numpy as np
import torch

# loading the dataset into a molecular database
molDB = ml.molecular_database.from_xyz_file('xyz.dat')
molDB.add_scalar_properties_from_file('en.dat', 'energy')
molDB.add_xyz_vectorial_properties_from_file('grad.dat', 'energy_gradients')

# spliting data with indices
np.random.seed(0)
idx = np.random.permutation(70099)
# idx = np.loadtxt('indices') # alternative way from saved file
subtrain_idx = idx[:56789]
validate_idx = idx[56789:63099]
test_idx = idx[63099:]

subtrain_molDB = molDB[subtrain_idx]
validate_molDB = molDB[validate_idx]
test_molDB = molDB[test_idx]

# define the weighing_function
def weighting_function(energy_reference, a):
    global_minimum = -284.33376035
    reference_tensor = torch.tensor(energy_reference - global_minimum)
    x=a*reference_tensor
    x_tensor = torch.tensor(x)
    x_pow5 = x_tensor ** 5
    x_pow4 = x_tensor ** 4
    x_pow3 = x_tensor ** 3
    w = -6 * x_pow5 + 15 * x_pow4 - 10 * x_pow3 + 1
    w = torch.clamp(w, min=0.000001)
    return w
    
# define the range of parameter a, and number of repeats
aStart = 0
aEnd = 5
aStep = 0.1
Nrepeat = 3 

# train the model with different a in the custom range
for a in np.arange(aStart, aEnd, aStep):
    for i in range(Nrepeat):
        ani = ml.models.ani(model_file=f"a{a}_repeat{i}.pt")
        ani.train(
            molecular_database=subtrain_molDB, 
            validation_molecular_database=validate_molDB, 
            property_to_learn='energy',
            xyz_derivative_property_to_learn='energy_gradients',
            energy_weighting_function=lambda en_ref: weighting_function(en_ref, a),
            hyperparameters={'lrReducePatience': 32}
        )

# Simulations
## Energy minima

In [ ]:
import mlatom as ml

minimum_confs = ml.molecular_database.from_xyz_file('Confs.xyz') # B3LYP energy minima from “Full-dimensional, ab initio potential energy surface for glycine with characterization of stationary points and zero-point energy calculations by means of diffusion Monte Carlo and semiclassical dynamics“, Conte et al.,  J. Chem. Phys. 153, 244301 (2020)

# load a ANI model
model = ml.models.ani(model_file='models/Best/pow_2.15.pt') # the best ANI model
# model = ml.models.ani(model_file='models/off_the_shelf/test1.pt') # the best off-the-shell model
# model = ml.models.ani(model_file='models/A/pow_1.2.pt') # the model A
# model = ml.models.ani(model_file='models/B/pow_0.3.pt') # the model B

# optimize geometries
optconfs = []
E_opt_minimum = []

for conf in minimum_confs:
    geomopt = ml.simulations.optimize_geometry(model=model,
                                    program='gaussian',
                                    initial_molecule=conf)
    optconfs.append(geomopt.optimized_molecule)
    E_opt_minimum.append(geomopt.optimized_molecule.energy)


E_delta_minimum = [(x - E_opt_minimum[0]) * ml.constants.Hartree2kcalpermol for x in E_opt_minimum]
print("relative minima in kcal/mol")
print(E_delta_minimum)

E_ref_minimum  = np.array([0.        , 0.58042116, 1.64245585, 1.27167305, 2.60883926,
                           4.9132172 , 5.83858032, 6.25032067]) # from the same literature

#calculate the mean absolute error
MAE_E_minimum = np.mean(np.abs(E_delta_minimum - E_ref_minimum))
print("MAE: minimum")
print(MAE_E_minimum)

# calculate frequencies
freqs=[]
for conf in optconfs:
    freq = ml.simulations.freq(model = model, molecule=conf, program='gaussian')
    freqs.append(conf.frequencies)
    print(conf.frequencies)

#DMC simulation
for conf in optconfs:
    dmc=ml.simulations.dmc(model=model, initial_molecule=conf)
    dmc.run(num_walkers=30000, num_timesteps=55000)
    print(f'ZPVE: {dmc.get_zpe(onwards=-1000, ret_cm=True) + 284.33355671 * 219474.63} cm-1')


## Transition states

In [1]:
import mlatom as ml

saddle_confs = ml.molecular_database.from_xyz_file('SPs.xyz') # B3LYP trainsition states from “Full-dimensional, ab initio potential energy surface for glycine with characterization of stationary points and zero-point energy calculations by means of diffusion Monte Carlo and semiclassical dynamics“, Conte et al.,  J. Chem. Phys. 153, 244301 (2020)

# load a ANI model
model = ml.models.ani(model_file='models/Best/pow_2.15.pt') # the best ANI model
# model = ml.models.ani(model_file='models/off_the_shelf/test1.pt') # the best off-the-shell model
# model = ml.models.ani(model_file='models/A/pow_1.2.pt') # the model A
# model = ml.models.ani(model_file='models/B/pow_0.3.pt') # the model B
    
# optimize geometries
optconfs = []
E_opt_TS = []
for conf in saddle_confs:
    geomopt = ml.optimize_geometry(model=model,
                                    program='gaussian',
                                    initial_molecule=conf,ts=True)
    optconfs.append(geomopt.optimized_molecule)
    E_opt_TS.append(geomopt.optimized_molecule.energy)

optconfs = ml.data.molecular_database(optconfs)

E_delta_TS = [(x - E_opt_minimum[0]) * ml.constants.Hartree2kcalpermol for x in E_opt_TS]
print("TSs: relative ener to gobal min. in kcal/mol")
print(E_delta_TS)

E_ref_TS = np.array([
2.216395955,
1.472736786,
5.020056782,
12.35143257,
13.72753083,
14.63501803,
10.37935309,
5.857495347,
8.114205005,
3.886126654,
14.57326086,
13.79186121,
15.91933828,
8.109630401,
6.318672678,
] )# from the same literature
                     
#calculate the mean absolute error
MAE_E_TS = np.mean(np.abs(E_delta_TS - E_ref_TS))
print("MAE: TS")
print(MAE_E_TS)

# calculate frequencies
freqs=[]
for conf in optconfs:
    freq = ml.simulations.freq(model = model, molecule=conf, program='gaussian')
    freqs.append(conf.frequencies)
    print(conf.frequencies)